In [1]:
import tensorflow as tf
from tensorflow import keras
import random
import math
import numpy as np

import graphsurgeon as gs
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

input_size_1 = 4
input_size_2 = 5
N_class = 4
N_epoch_WDGRL = 50
N_epoch_STDAN = 10
acpt_thres_opt = 0.5

sbj_idx = 0;

In [2]:
# Net = "WDGRL_"
Net = "STDAN_"

if Net == "WDGRL_":
    Filename = Net + "C_sbj" + str(sbj_idx) + "_iter" + str(N_epoch_WDGRL)
elif Net == "STDAN_":
    Filename = Net + "C_sbj" + str(sbj_idx) + "_Acpt_thres_" + '%.1f' % acpt_thres_opt + "_iter" + str(N_epoch_STDAN)
model = tf.keras.models.load_model("Weights/h5/" + Filename + ".h5")

# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda inputs: model(inputs))
full_model = full_model.get_concrete_function([tf.TensorSpec(model_input.shape, model_input.dtype) for model_input in model.inputs])
# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)    
frozen_func.graph.as_graph_def() #Returns a serialized GraphDef representation of this graph.

layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 60)
print("Frozen model layers: ")
for layer in layers:
    print(layer)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)

def add_plugin(graph):
    # Change AddV2
    #for node in graph.find_nodes_by_op('AddV2'):
    #    gs.update_node(node, op='Add')
    
    # Remove NoOp
    for node in graph.find_nodes_by_op('NoOp'):
        print("Find the NoOp layer")
        gs.update_node(node, op='Identity')

    return graph

dynamic_graph = gs.DynamicGraph(frozen_func.graph)
modified_graph = add_plugin(dynamic_graph)
modified_graph.find_nodes_by_op("NoOp")

############################# SAVE PB FILE #############################

# name of the .pb file
frozen_graph_filename = "Weights/pb/" + Filename + ".pb"

# Save frozen graph to disk
pb_model = tf.io.write_graph(graph_or_graph_def=modified_graph.as_graph_def(),
                  logdir='',
                  name=frozen_graph_filename,
                  as_text=False)

------------------------------------------------------------
Frozen model layers: 
inputs
model_1/conv2d/Conv2D/ReadVariableOp/resource
model_1/conv2d/Conv2D/ReadVariableOp
model_1/conv2d/Conv2D
model_1/conv2d/BiasAdd/ReadVariableOp/resource
model_1/conv2d/BiasAdd/ReadVariableOp
model_1/conv2d/BiasAdd
model_1/batch_normalization/ReadVariableOp/resource
model_1/batch_normalization/ReadVariableOp
model_1/batch_normalization/ReadVariableOp_1/resource
model_1/batch_normalization/ReadVariableOp_1
model_1/batch_normalization/FusedBatchNormV3/ReadVariableOp/resource
model_1/batch_normalization/FusedBatchNormV3/ReadVariableOp
model_1/batch_normalization/FusedBatchNormV3/ReadVariableOp_1/resource
model_1/batch_normalization/FusedBatchNormV3/ReadVariableOp_1
model_1/batch_normalization/FusedBatchNormV3
model_1/activation/Relu
model_1/conv2d_1/Conv2D/ReadVariableOp/resource
model_1/conv2d_1/Conv2D/ReadVariableOp
model_1/conv2d_1/Conv2D
model_1/conv2d_1/BiasAdd/ReadVariableOp/resource
model_1/conv